<a href="https://colab.research.google.com/github/DanielAcostaRoa/B-COSTORCH/blob/master/COSTORCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
!pip install kornia
import kornia

In [ ]:
torch.manual_seed(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
name = "/content/imp1_3k.jpg"
image = cv2.imread(name,0)
print(image.shape)

(324, 452)


In [ ]:
class BCOSFIRE:
  def __init__(self,w,l,sizeStep,nRotaciones,sigma0,alpha):
    self.sigmaDoG = w / 1.92
    self.rho = np.arange(0,int(l/2)+1,sizeStep)
    self.phi = [np.pi/2.0,np.pi/2.0+np.pi]
    self.sigma0 = sigma0
    self.alpha = alpha    
    self.sigmaBlur = self.rho*self.alpha + self.sigma0
    self.discrInvRot = 6
    self.sizeDoG = int(max(self.sigmaDoG))*6+1 if int(max(self.sigmaDoG))*6+1 <= 25 else 25
    self.sizeBlur = int(max(self.sigmaBlur))*6+1 if int(max(self.sigmaBlur))*6+1 <=25 else 25
    self.convDoGKernels = torch.stack([self.getDoGKernel(self.sizeDoG,sig) for sig in self.sigmaDoG],0)
    self.convBlurKernels= torch.stack(len(self.sigmaDoG)*[self.getGaussianKernel(self.sizeBlur,sig) for sig in self.sigmaBlur],1)
    self.convShiftKernels = [(self.getAffineTransformation(self.phi[0]+i*(np.pi/self.discrInvRot)),self.getAffineTransformation(self.phi[1]+i*(np.pi/self.discrInvRot))) for i in range(self.discrInvRot)]

  def getDoGKernel(self,ksize,sig):
    gaussianKernel_0 = kornia.get_gaussian_kernel2d((ksize, ksize), (0.5*sig, 0.5*sig))
    gaussianKernel_1 = kornia.get_gaussian_kernel2d((ksize, ksize), (sig, sig))
    return torch.unsqueeze(gaussianKernel_0.float()-gaussianKernel_1.float(),0)
  
  def getGaussianKernel(self,ksize,sig):
    return torch.unsqueeze(kornia.get_gaussian_kernel2d((ksize, ksize), (sig, sig)).float(),0)

  def shift_bankImages(self,imageBank,idx_angInv,idx_angShift):
    return kornia.warp_affine(imageBank,self.convShiftLayers[idx_angInv][idx_angShift],(imageBank.shape[3],imageBank.shape[4]))
  
  def geometricMean(self,shiftImageBank):
    producto = torch.prod(shiftImageBank,1)
    producto.pow_(shiftImageBank.shape[1])
    return producto

  def getAffineTransformation(self,angulo):
    transf_matrix = torch.zeros(len(self.rho),2,3)
    for i in range(len(self.rho)):
      transf_matrix[i][0][0] = 1
      transf_matrix[i][1][1] = 1
      transf_matrix[i][0][2] = -self.rho[i]*np.cos(angulo)
      transf_matrix[i][1][2] = self.rho[i]*np.sin(angulo)
    return transf_matrix

  def multiscaleResponse(self,batchBankImage):
    response = torch.zeros(batchBankImage.shape[0],len(self.sigma),batchBankImage.shape[2],batchBankImage.shape[3])
    for im in range(len(batchBankImage)):
      for i in range(self.discrInvRot):
        shift0 = shift_bankImages(batchBankImage[im].view(-1,len(self.rho),batchBankImage[im].shape[2],batchBankImage[im].shape[3]).transpose(0,1),i,0)
        shift1 = shift_bankImages(batchBankImage[im].view(-1,len(self.rho),batchBankImage[im].shape[3],batchBankImage[im].shape[4]).transpose(0,1),i,0)
        shiftBank = torch.stack((shift0.transpose(0,1),shift1.transpose(0,1)),1)
        torch.max(response[im],self.geometricMean(shiftBank.view()))
    return response

  def BCOSFIRE_Response(self,inputImage):
    img = torch.tensor_like(inputImage)
    F.normalize(inputImage,p=1,dim=1,out=img)
    img = torch.unsqueeze(img.float(), dim=0)  # Bx1xHxW
    dog_response = F.conv2d(img,self.convDoGKernels, bias=False, padding = (self.sizeDoG-1)/2.0)
    F.relu(dog_response,inplace=True)
    blur_response = F.conv2d(dog_response,self.convBlurKernels, groups = len(self.sigmaDoG), bias=False, padding = (self.sizeBlur-1)/2.0)
    F.relu(blur_response,inplace=True)
    filter_response = self.multiscaleResponse(blur_response)
    return filter_resonse


In [ ]:
filtro1 = BCOSFIRE(np.array([7]),30,1,6,0.1,0.9)

In [ ]:
filtro1.convDoGKernels.shape

torch.Size([1, 1, 19, 19])

In [ ]:
filtro1.convBlurKernels.shape

torch.Size([1, 16, 25, 25])

In [ ]:
filtro1.rho

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [ ]:
filtro1.convShiftKernels[0][1].shape

torch.Size([16, 2, 3])

In [ ]:
listA = [0]
listB = listA
listB = [0,1]
listB.append(1)
print(listA)

[0]


In [ ]:
def RUSTICO(imagen,w,l,n,sigma0,alpha,la,eps,epsA,discr=12):
  resp_P, banc_P = mult_bcosfire(v,w,l,n,sigma0[0],alpha[0],discr,0)
  resp_N, banc_N = mult_bcosfire(v,la*w,l,n,sigma0[1],alpha[1],discr,1)
  rows,cols = imagen.shape
  resp_RUSTICO = []
  respF = np.zeros((rows,cols))
  for i in range(discr):
    newR = np.maximum(0, epsA*banc_P[i] - eps*banc_N[i])
    resp_RUSTICO.append(newR)
    respF= np.maximum(respF,newR)
  return respF, resp_RUSTICO, resp_P, resp_N

In [ ]:
r, b, rp, rn = RUSTICO(v,13,51,1, (0.0,0.0), (1.1,1.1), 5.1, 2.5, 1.0, 8 )
metr = metr1_ROC(r,gt)

In [ ]:
cv2_imshow(r)

In [ ]:
v_filtrada = Kuwahara(v,5)
cv2_imshow(v_filtrada)

In [ ]:
r1, b1, rp1, rn1 = RUSTICO(v_filtrada, 15, 71, 1, (0.0,0.0), (0.9,0.9), 8.1, 1.1, 1.0, 8 )
metr = metr1_ROC(r1,gt) 

In [ ]:
cv2_imshow(r1)